In [1]:
#Install dependencies
%pip install ultralytics opencv-python-headless

  Using cached opencv_python_headless-4.11.0.86-cp37-abi3-win_amd64.whl (39.4 MB)
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\amanp\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [2]:
# import modules
import os
import cv2
import torch
import uuid
from ultralytics import YOLO
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

In [4]:
#Load trained segmentation model
model = YOLO(r"runs\segment\yolov8_food_segmentation\weights\best.pt")

In [5]:
input_dir = "input_images"
output_dir = "output_items"
os.makedirs(input_dir, exist_ok=True)
os.makedirs(output_dir, exist_ok=True)

In [9]:
def compute_iou(mask1, mask2):
    intersection = np.logical_and(mask1, mask2).sum()
    union = np.logical_or(mask1, mask2).sum()
    return intersection / union if union > 0 else 0

In [10]:
#Function to process images and crop segmented food items
def segment_and_save_items(image_path, output_dir, iou_thresh=0.5):
    results = model(image_path)[0]
    img = cv2.imread(image_path)

    if results.masks is None:
        print(f"No objects found in {image_path}")
        return

    masks = results.masks.data.cpu().numpy()
    boxes = results.boxes.xyxy.cpu().numpy()
    classes = results.boxes.cls.cpu().numpy()

    kept_masks = []
    kept_boxes = []
    kept_classes = []

    for i, mask in enumerate(masks):
        is_duplicate = False
        for kept_mask in kept_masks:
            iou = compute_iou(mask.astype(bool), kept_mask.astype(bool))
            if iou > iou_thresh:
                is_duplicate = True
                break
        if not is_duplicate:
            kept_masks.append(mask)
            kept_boxes.append(boxes[i])
            kept_classes.append(classes[i])

    for mask, box, class_id in zip(kept_masks, kept_boxes, kept_classes):
        class_name = model.names[int(class_id)]

        x1, y1, x2, y2 = map(int, box)
        masked_img = cv2.bitwise_and(img, img, mask=mask.astype(np.uint8))
        cropped = masked_img[y1:y2, x1:x2]

        if cropped.size == 0 or cropped.shape[0] < 20 or cropped.shape[1] < 20:
            continue

        filename = f"{class_name}_{uuid.uuid4().hex[:8]}.jpg"
        cv2.imwrite(os.path.join(output_dir, filename), cropped)
        print(f"Saved: {filename}")

In [14]:
# Run segmentation on all images in the input folder
for filename in os.listdir(input_dir):
    if filename.lower().endswith(('.jpg', '.png', '.jpeg')):
        image_path = os.path.join(input_dir, filename)
        segment_and_save_items(image_path, output_dir)


image 1/1 c:\Users\amanp\OneDrive\COLLEGE\Major Project - Health-Mate\HealthMateCodeSpace\input_images\indian-vegetarian-thali-5_jpg.rf.e86fcb07c0a053824a68b45a0a892247.jpg: 640x640 2 bowls, 3 breads, 6 currys, 1 rice, 1 salad, 3 vegitable_mixs, 472.1ms
Speed: 11.8ms preprocess, 472.1ms inference, 46.4ms postprocess per image at shape (1, 3, 640, 640)
Saved: salad_945b4d11.jpg
Saved: curry_4bc292c6.jpg
Saved: curry_30499a8f.jpg
Saved: rice_e5715ed8.jpg
Saved: bread_478a71db.jpg
Saved: curry_49e458f6.jpg
Saved: vegitable_mix_0acbb0f1.jpg
Saved: bread_2ba79186.jpg
Saved: bread_b3a7b61c.jpg
